In [34]:
%load_ext autoreload
%autoreload 2

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval.data.synth_data_generator import generate_data
from mmm_eval.configs import PyMCConfig
from pymc_marketing.prior import Prior

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate data

In [2]:
data = generate_data()
data.to_csv("data.csv", index=False)

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

# Fit a PyMC Model

In [35]:
base_model = MMM(
    date_column="date_week" ,
    channel_columns=["channel_1","channel_2"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

base_model.fit(X=X, y=y, chains=4, target_accept=0.85)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, adstock_alpha, saturation_lam, saturation_beta, y_sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 5 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data

# Create a config

To run the evaluation suite, we need to store the configuration used to create the original model.

To do this, we create a `PyMCConfig` object

In [36]:
# We dont need X and y - we'll get those from the input data!
fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

base_config = PyMCConfig(base_model, fit_kwargs=fit_kwargs, response_column="quantity", revenue_column="revenue")

In [37]:
#base_config.model_config.config

# Check that we can use the config to fit a PyMC MMM model

In [38]:
m1 = MMM(**base_config.model_config.config)
m1.fit(X=X, y=y,**base_config.fit_config.config)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, adstock_alpha, saturation_lam, saturation_beta, y_sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 5 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data

# Test a more complex config

In [39]:
total_spend_per_channel = data[["channel_1", "channel_2"]].sum(axis=0)

spend_share = total_spend_per_channel / total_spend_per_channel.sum()
n_channels = 2
prior_sigma = n_channels * spend_share.to_numpy()

model_config = my_model_config = {
    "intercept": Prior("Normal", mu=0.5, sigma=0.2),
    "saturation_beta": Prior("HalfNormal", sigma=prior_sigma),
    "gamma_control": Prior("Normal", mu=0, sigma=0.05),
    "gamma_fourier": Prior("Laplace", mu=0, b=0.2),
}

config = {
    "date_column": "date_week",
    "channel_columns": ["channel_1", "channel_2"],
    "control_columns": ["price", "event_1", "event_2"],
    "adstock": GeometricAdstock(l_max=4),  # or DelayedAdstock(), etc.
    "saturation": LogisticSaturation(),
    "yearly_seasonality": 2,
    "model_config": model_config,
}

fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

# fit another base model
m2 = MMM(**config)
m2.fit(X=X, y=y, **fit_kwargs)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, adstock_alpha, saturation_lam, saturation_beta, gamma_control, gamma_fourier, y_sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 15 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data

In [53]:
from mmm_eval.adapters.experimental.pymc import PyMCAdapter

config = PyMCConfig(m2, fit_kwargs=fit_kwargs, response_column="quantity", revenue_column="revenue")
adapter = PyMCAdapter(config)

adapter.fit(data)

Control column 'price' has values outside [0, 1] range: min=5.0003, max=6.9575. Consider scaling this column to 0-1 range as per PyMC best practices.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, adstock_alpha, saturation_lam, saturation_beta, gamma_control, gamma_fourier, y_sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 14 seconds.


In [54]:
preds = adapter.predict(data)
len(preds)

Control column 'price' has values outside [0, 1] range: min=5.0003, max=6.9575. Consider scaling this column to 0-1 range as per PyMC best practices.
Sampling: [y]


Output()

179

In [55]:
adapter.get_channel_roi()

channel_1      17.015036
channel_2    2758.500141
dtype: float64

In [66]:
import pandas as pd

pd.Timestamp(adapter.trace["observed_data"]["date"].max().item())

Timestamp('2021-08-30 00:00:00')

In [ ]:
config1 = PyMCConfig(m1, fit_kwargs=fit_kwargs, response_column="quantity", revenue_column="revenue")

In [ ]:
config1.model_config.__dict__

In [ ]:


# m2 = MMM(**config1.model_config.config)
# m2.fit(X=X, y=y, **config1.fit_config.config)

# Save the config to JSON

In [ ]:
config1.save_config(save_path="./", file_name="config1")

# And load it in a new instantiation

In [ ]:
loaded_config = PyMCConfig().load_config("config1.json")
loaded_config.model_config.config

# Check if we can run the model from the loaded config

In [ ]:
m3 = MMM(**loaded_config.model_config.config)
m3.fit(X=X, y=y, **loaded_config.fit_config.config)